# Getting milliyetNER train data and parsing sentences for building hidden markov model

In [1]:
import pandas as pd
import numpy as np

words = []
entity = []
sentences = []
tokens = []
entitys = []
with open("./train.txt", encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        if line=='\n':
            sentences.append(list(zip(words, entity)))
            words=[]
            entity=[]
            continue
        line_split = line.split()
        tokens.append(line_split[0])
        entitys.append(line_split[1])
        words.append(line_split[0])
        entity.append(line_split[1])


# Creating probability matrix train data

In [2]:


tokens = [token.lower() for token in tokens]
# print(tokens[0])
# print(entitys)

df = pd.DataFrame(tokens,entitys).reset_index().rename(columns={0:'tokens','index':'entitys'})
df['values']=1
df = df.pivot_table(index='entitys', columns='tokens', aggfunc='sum').fillna(0)
df = df.div(df.sum(axis=1), axis=0)
df.columns = df.columns.droplevel(0)

print(df)

tokens                 "         $         %         &         '         (  \
entitys                                                                      
B-LOCATION      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
B-ORGANIZATION  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
B-PERSON        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
I-LOCATION      0.000000  0.000000  0.000000  0.000000  0.003115  0.000000   
I-ORGANIZATION  0.000000  0.000000  0.000000  0.002730  0.004949  0.000000   
I-PERSON        0.000634  0.000000  0.000000  0.000000  0.001267  0.000000   
O               0.023062  0.000003  0.000027  0.000024  0.046473  0.003584   

tokens                )         +         ,         -  ...   şüpheyi  \
entitys                                                ...             
B-LOCATION      0.00000  0.000000  0.000000  0.000000  ...  0.000000   
B-ORGANIZATION  0.00000  0.000000  0.000000  0.000000  ...  0.000000   
B-PERSON 

# Creating transition probability matrix

In [3]:
df2 = []

for sentence in sentences:
    for index in range(len(sentence)-1):
        first_token = sentence[index][1]
        next_token = sentence[index+1][1]
        df2.append({'first_token':first_token,'next_token':next_token})

        
        
df2 = pd.DataFrame(df2)
df2['values'] = 1
df2 = df2.pivot_table(index='first_token',columns='next_token',aggfunc='sum').fillna(0)
df2 = df2.div(df2.sum(axis=1), axis=0)
df2.columns = df2.columns.droplevel(0)

df2 = pd.concat([pd.DataFrame(columns=['B-LOCATION','B-ORGANIZATION','B-PERSON','I-LOCATION','I-ORGANIZATION','I-PERSON','O']
                              ,index=['<s>'],data=[[5/7,5/7,5/7,0,0,0,2/7]]),df2],axis=0)

print(df2)

                B-LOCATION  B-ORGANIZATION  B-PERSON  I-LOCATION  \
<s>               0.714286        0.714286  0.714286    0.000000   
B-LOCATION        0.006657        0.001985  0.000467    0.105103   
B-ORGANIZATION    0.008235        0.002786  0.000484    0.000000   
B-PERSON          0.002650        0.001363  0.005982    0.000000   
I-LOCATION        0.007843        0.000784  0.003137    0.301176   
I-ORGANIZATION    0.004281        0.002568  0.000856    0.000000   
I-PERSON          0.003838        0.003338  0.001502    0.000000   
O                 0.019638        0.017229  0.029171    0.000000   

                I-ORGANIZATION  I-PERSON         O  
<s>                   0.000000  0.000000  0.285714  
B-LOCATION            0.000000  0.000000  0.885788  
B-ORGANIZATION        0.386702  0.000000  0.601792  
B-PERSON              0.000000  0.434878  0.555126  
I-LOCATION            0.000000  0.000000  0.687059  
I-ORGANIZATION        0.456678  0.000000  0.535616  
I-PERSON        

# getting test data for test our model and get prediction values

In [4]:

test_tokens = []
test_entites = []
with open("./test.txt", encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        if line=='\n':
#             sentences.append(list(zip(words, entity)))
#             words = []
#             entity = []
            continue
        line_split = line.split()
        test_tokens.append(line_split[0])
        test_entites.append(line_split[1])

list_of_entity = []
previous = '<s>'

tokens = [token.lower() for token in test_tokens]

for token in tokens:
    try:
        df[token]
    except:
        list_of_entity.append((token, 'O'))
        continue
    matrix = df[token].multiply(df2.loc[previous])
    previous = df.index[np.argmax(matrix)]
    token_label_pair = (token,previous)
    list_of_entity.append(token_label_pair)

In [5]:
predicted_entities=[]
for i in list_of_entity:
    predicted_entities.append(i[1])

# Comparing true POS tags with predicted ones without 'O' 

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics

labels = ['B-PERSON',
 'I-PERSON',
 'B-LOCATION',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-LOCATION']

metrics_table_test = metrics.classification_report(test_entites, predicted_entities, labels=labels)
print(metrics_table_test)

                precision    recall  f1-score   support

      B-PERSON       0.89      0.75      0.82      1603
      I-PERSON       0.91      0.52      0.67       803
    B-LOCATION       0.79      0.86      0.83      1126
B-ORGANIZATION       0.85      0.55      0.67       873
I-ORGANIZATION       0.63      0.15      0.24       864
    I-LOCATION       0.76      0.34      0.47       211

     micro avg       0.84      0.60      0.70      5480
     macro avg       0.81      0.53      0.61      5480
  weighted avg       0.82      0.60      0.67      5480



# Getting dev.txt data to predict values and getting compare results

In [7]:
test_tokens = []
test_entites = []
with open("./dev.txt", encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        if line=='\n':
#             sentences.append(list(zip(words, entity)))
#             words = []
#             entity = []
            continue
        line_split = line.split()
        test_tokens.append(line_split[0])
        test_entites.append(line_split[1])

list_of_entity = []
previous = '<s>'

tokens = [token.lower() for token in test_tokens]

for token in tokens:
    try:
        df[token]
    except:
        list_of_entity.append((token, 'O'))
        continue
    matrix = df[token].multiply(df2.loc[previous])
    previous = df.index[np.argmax(matrix)]
    token_label_pair = (token,previous)
    list_of_entity.append(token_label_pair)
    
predicted_entities=[]
for i in list_of_entity:
    predicted_entities.append(i[1])

metrics_table_dev = metrics.classification_report(test_entites, predicted_entities, labels=labels)
print(metrics_table_dev)

                precision    recall  f1-score   support

      B-PERSON       0.90      0.86      0.88      1400
      I-PERSON       0.90      0.68      0.78       680
    B-LOCATION       0.81      0.92      0.86       942
B-ORGANIZATION       0.87      0.66      0.75       842
I-ORGANIZATION       0.68      0.26      0.37       589
    I-LOCATION       0.68      0.54      0.60       107

     micro avg       0.85      0.72      0.78      4560
     macro avg       0.81      0.65      0.71      4560
  weighted avg       0.84      0.72      0.76      4560



# Conclusion
I did get slightly better results in dev.txt data. I found some typing error in test.txt and dev.txt data and I added try block code for that reason. It catches typos. Maybe dev.txt data has less typo than test.txt.

# Writing metric table results

In [8]:
with open('metric_table_test.txt', 'w') as f:
    f.write(metrics_table_test)
with open('metric_table_dev.txt', 'w') as f:
    f.write(metrics_table_dev)